In [1]:
using Plots;
gadfly();

In [2]:
include("../fdtd/update.jl");
include("../fdtd/sources.jl");
include("../fdtd/boundaries.jl");
using update;
using sources;

In [3]:
#Global parameters
size = 200;
endTime = 300;
num_snaps = 300;
snap_step = div(endTime, num_snaps);
#Grid
hy = zeros(size);
ez = zeros(size);
# output params
ez_snapshot = Array{Any}(num_snaps);
hy_snapshot = Array{Any}(num_snaps);

In [4]:
# Time steps

for time in 1:endTime
    # Incident
    # ez_inc, hy_inc = sources.gaussian_source(50, time);
    delay = 30.
    width = 100.
    
    ez_inc = exp(-(time + 0.5 - (-0.5) - delay) * (time + 0.5 - (-0.5) - delay) / width);
    hy_inc = exp(-(time - delay) * (time - delay) / width) / globals.imp0;
    
    #
    # Magnetic
    #
    
    # ABC
    boundaries.trivial_abc!(hy, size);
    
    # Interior update
    update.update_magnetic_field!(ez, hy);    
    
    # TFSF
    hy[49] -= hy_inc 
    
    #
    # Electric
    #
    
    # ABC
    boundaries.trivial_abc!(ez, 1, false);    
    
    # Interior update
    update.update_electric_field!(ez, hy);
    
    # TFSF
    ez[50] += ez_inc 
 
    # Snapshots for animation
    if mod(time, snap_step) == 0
        ez_snapshot[div(time,snap_step)] = (time, copy(ez))
        hy_snapshot[div(time,snap_step)] = (time, copy(hy).*globals.imp0)        
    end
    
end

In [5]:
anim = Animation()

for i = 1:num_snaps
    p = plot(1:size, ez_snapshot[i][2], lab="Ez")
    plot!(1:size, hy_snapshot[i][2], lab="Hy*imp0")
    
    time = ez_snapshot[i][1]
    plot!(ann=[(150, 1.5, "time =$time")])
    plot!(ann=[(0, 1.1, "ABC")])
    plot!(ann=[(180, 1.1, "ABC")])    
    plot!(xlims=(1, 200), ylims=(-2, 2))
    frame(anim, p)
end
gif(anim, "./04_tfsf.gif", fps=15)

[Plots.jl] Initializing backend: gadfly


INFO: Saved animation to /media/storage/Documents/Github/1d-fdtd/examples/04_tfsf.gif


Plots.AnimatedGif("/media/storage/Documents/Github/1d-fdtd/examples/04_tfsf.gif")